# Parsing minified pipelines (`FROM`: key)

Letting the C-PAC pipeline configuration code parse minified pipelines can save time rather than tracing a chain of `FROM` imports.

First we'll create a little function to
* take a path to a YAML file as a positional argument or a raw YAML string as a keyword argument,
* print the full config with default comments as YAML,
  and
* return the Configuration object.

In [1]:
from pathlib import Path
from typing import Optional
import yaml
from CPAC.utils.configuration import Configuration, Preconfiguration
from CPAC.utils.configuration.yaml_template import create_yaml_from_template


def show_full_config(path_to_config: Optional[Path | str] = None, *, full_yaml: Optional[str] = None) -> dict:
    """Given a path to a minified C-PAC participant pipeline configuration, return the full loaded config."""
    # load the config
    if path_to_config:
        if isinstance(path_to_config, str):
            path_to_config = Path(path_to_config)
        with path_to_config.open("r", encoding="utf-8") as _config_file:
            full_yaml = _config_file.read()
    full_configuration = Configuration(yaml.safe_load(full_yaml))
    # display the config
    print(create_yaml_from_template(full_configuration, "default", skip_env_check=True))
    # return the config
    return full_configuration

First we'll load and print the `fmriprep-options` preconfig by just importing it and making no changes:

In [2]:
fmriprep_options = show_full_config(full_yaml="FROM: fmriprep-options")

%YAML 1.1
---
# CPAC Pipeline Configuration YAML file
# Version 1.8.7.dev1
#
# http://fcp-indi.github.io for more info.
#
# Tip: This file can be edited manually with a text editor for quick modifications.
pipeline_setup:

  # Name for this pipeline configuration - useful for identification.
  # This string will be sanitized and used in filepaths
  pipeline_name: cpac_fmriprep-options
  output_directory:

    # Quality control outputs
    quality_control:

      # Generate eXtensible Connectivity Pipeline-style quality control files
      generate_xcpqc_files: Off

      # Generate quality control pages containing preprocessing and derivative outputs.
      generate_quality_control_images: Off

    # Directory where C-PAC should write out processed data, logs, and crash reports.
    # - If running in a container (Singularity/Docker), you can simply set this to an arbitrary
    #   name like '/outputs', and then map (-B/-v) your desired output directory to that label.
    # - If running

We'll confirm it's the same as loading the preconfig directly:

In [3]:
fmriprep_options - Preconfiguration("fmriprep-options")

{}

In [4]:
Preconfiguration("fmriprep-options") - fmriprep_options

{}

Okay, good, no difference in either direction.

Now we can try loading a file with some modifications:

In [5]:
priors_path_config = Path("priors_path.yml")
with priors_path_config.open("r", encoding="utf-8") as _file:
    print(_file.read())

FROM: fmriprep-options
segmentation:
  tissue_segmentation:
    FSL-FAST:
      use_priors:
        priors_path: /custom_fsl/data/standard/tissuepriors/2mm



In [6]:
priors_path = show_full_config(priors_path_config)

%YAML 1.1
---
# CPAC Pipeline Configuration YAML file
# Version 1.8.7.dev1
#
# http://fcp-indi.github.io for more info.
#
# Tip: This file can be edited manually with a text editor for quick modifications.
pipeline_setup:

  # Name for this pipeline configuration - useful for identification.
  # This string will be sanitized and used in filepaths
  pipeline_name: cpac_fmriprep-options
  output_directory:

    # Quality control outputs
    quality_control:

      # Generate eXtensible Connectivity Pipeline-style quality control files
      generate_xcpqc_files: Off

      # Generate quality control pages containing preprocessing and derivative outputs.
      generate_quality_control_images: Off

    # Directory where C-PAC should write out processed data, logs, and crash reports.
    # - If running in a container (Singularity/Docker), you can simply set this to an arbitrary
    #   name like '/outputs', and then map (-B/-v) your desired output directory to that label.
    # - If running

And we can see that our modification is the only difference:

In [7]:
priors_path - fmriprep_options

{'segmentation': {'tissue_segmentation': {'FSL-FAST': {'use_priors': {'priors_path': ('/custom_fsl/data/standard/tissuepriors/2mm', None)}}}}}

In [8]:
fmriprep_options - priors_path

{'segmentation': {'tissue_segmentation': {'FSL-FAST': {'use_priors': {'priors_path': (None, '/custom_fsl/data/standard/tissuepriors/2mm')}}}}}

(The 2-tuple for each nested key in a `DiffDict` is (the value in the `Configuration` left of the `-`, the value in the `Configuration` right of the `-`) 

In [9]:
(priors_path - fmriprep_options)["segmentation"]["tissue_segmentation"]["FSL-FAST"]["use_priors"]["priors_path"]

('/custom_fsl/data/standard/tissuepriors/2mm', None)

In [10]:
priors_path["segmentation", "tissue_segmentation", "FSL-FAST", "use_priors", "priors_path"]

'/custom_fsl/data/standard/tissuepriors/2mm'

In [11]:
fmriprep_options["segmentation", "tissue_segmentation", "FSL-FAST", "use_priors", "priors_path"]

In [12]:
type(fmriprep_options["segmentation", "tissue_segmentation", "FSL-FAST", "use_priors", "priors_path"])

NoneType

You can paste some raw YAML between the `"""`s below to load a full config and try it out yourself:

In [ ]:
pasted_full_config = show_full_config(full_yaml="""
""")